In [50]:
import json
import os
from os import path
import sys
from collections import defaultdict

sys.path.append('/home/shtoshni/Research/long-doc-coref/src')

from copy import deepcopy

from coref_utils.utils import get_mention_to_cluster_idx, remove_singletons

In [51]:
log_file = "/home/shtoshni/Research/long-doc-coref/models/umem_singleton_round_2/train.log.jsonl"

output_dir = "/home/shtoshni/Research/litbank_coref/data/ontonotes/ment_singletons"
# output_dir = "/tmp"

In [52]:
data = []
with open(log_file) as f:
    for line in f:
        instance = json.loads(line.strip())
        data.append(instance)

data = remove_singletons(data)

other_mentions = []

num_gt_mentions = 0
for instance in data:
    doc_key = instance["doc_key"]
    gt_mentions = get_mention_to_cluster_idx(instance["clusters"])
    num_gt_mentions += len(gt_mentions)
    pred_mentions = instance["pred_mentions"]
    mention_scores = instance["mention_scores"]
    for pred_mention, mention_score in zip(pred_mentions, mention_scores):
        if tuple(pred_mention) not in gt_mentions:
            other_mentions.append((mention_score, doc_key, pred_mention))
            
            
other_mentions = sorted(other_mentions, key=lambda x: x[0], reverse=True)

In [53]:
print(len(other_mentions))
print(num_gt_mentions)

282923
155558


In [54]:
if not path.exists(output_dir):
    os.makedirs(output_dir)
    


for num_mentions in [30 * 1e3, 60 * 1e3, 90 * 1e3, 120 * 1e3, 150 * 1e3]:
    singleton_clusters = defaultdict(list)
    for _, doc_key, mention in other_mentions[:int(num_mentions)]:
        singleton_clusters[doc_key].append([mention])
    
    ment_file = path.join(output_dir, f'{int(num_mentions/1000)}.jsonlines')
    print(ment_file)
    
    with open(ment_file, 'w') as f:
        f.write(json.dumps(singleton_clusters) + "\n")
        
        
    

/tmp/30.jsonlines
/tmp/60.jsonlines
/tmp/90.jsonlines
/tmp/120.jsonlines
/tmp/150.jsonlines
